In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

### Launch machines

In [ ]:
import enoslib as en
from grid5000 import Grid5000
from pathlib import Path

conf_file = Path.home() / ".python-grid5000.yaml"
gk = Grid5000.from_yaml(conf_file)
gk.sites.list() # if credentials were written adequatly in .env, this should work

job_name = 'dev'

conf = (
    en.G5kConf.from_settings(
        job_name=job_name,
        job_type=[],#"deploy"],
        # key="/home/adrien/.ssh/grid",
        #env_name="ubuntu2204-min",
        walltime="6:00:00"
    )    
    .add_machine(roles=["client"], cluster="uvb", nodes=1)
    .add_machine(roles=["server"], cluster="uvb", nodes=1)
    # .add_machine(roles=["client"], servers=["uvb-25.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["server"], servers=["uvb-26.sophia.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["client"], servers=["grimoire-3.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
    # .add_machine(roles=["server"], servers=["grimoire-4.nancy.grid5000.fr"]) #cluster="grimoire", nodes=1)
)

provider = en.G5k(conf)

roles, networks = provider.init()

### Docker 



In [ ]:
results = en.run_command("sudo-g5k apt update", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt install -y apt-transport-https ca-certificates curl software-properties-common", roles=roles["client"]+roles["server"])
results = en.run_command("curl -fsSL https://download.docker.com/linux/debian/gpg | sudo-g5k gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg", roles=roles["client"]+roles["server"])
results = en.run_command('echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/debian $(lsb_release -cs) stable" | sudo-g5k tee /etc/apt/sources.list.d/docker.list > /dev/null', roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt update", roles=roles["client"]+roles["server"])
results = en.run_command("apt-cache policy docker-ce", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt install -y docker-ce", roles=roles["client"]+roles["server"])


### Bazel

### Envoy - FIRST TIME

In [ ]:
results = en.run_command("sudo-g5k apt install apt-transport-https curl gnupg -y", roles=roles["client"]+roles["server"])
results = en.run_command("curl -fsSL https://bazel.build/bazel-release.pub.gpg | gpg --dearmor >bazel-archive-keyring.gpg", roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k mv bazel-archive-keyring.gpg /usr/share/keyrings", roles=roles["client"]+roles["server"])
results = en.run_command('echo "deb [arch=amd64 signed-by=/usr/share/keyrings/bazel-archive-keyring.gpg] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo-g5k tee /etc/apt/sources.list.d/bazel.list', roles=roles["client"]+roles["server"])
results = en.run_command("sudo-g5k apt update && sudo-g5k apt install -y bazel", roles=roles["client"]+roles["server"])
#results = en.run_command("sudo-g5k apt update && sudo-g5k apt full-upgrade -y", roles=roles["client"])

In [ ]:
results = en.run_command("cd /home/mdeneuville && git clone https://github.com/adribr53/envoy", roles=roles["client"])
results = en.run_command("cd /home/mdeneuville/envoy && ./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["client"])
#results = en.run_command("./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["server"], background=True)
#results = en.run_command("cp -r /tmp/envoy-docker-build/ /home/agiot/", roles=roles["client"])
# copy /root/envoy to /home/agiot

### Envoy - Save build

In [ ]:
results = en.run_command("cd /tmp && tar -czvf envoy-docker-build.tar.gz envoy-docker-build/", roles=roles["client"])
results = en.run_command("scp -r /tmp/envoy-docker-build.tar.gz rdma-vm:.g5k", roles=roles["client"])

### Envoy - Get build back

In [ ]:
# results = en.run_command("scp -r rdma-vm:./g5k/envoy-docker-build.tar.gz /tmp", roles=roles["client"]+roles["server"])
results = en.run_command("cp /home/mdeneuville/envoy-docker-build.tar.gz /tmp", roles=roles["client"]+roles["server"])
results = en.run_command("cd /tmp && sudo-g5k tar -xzvf envoy-docker-build.tar.gz", roles=roles["client"]+roles["server"])
results = en.run_command("cd /tmp && sudo chown -R root:docker envoy-docker-build && sudo chmod 755 -R envoy-docker-build", roles=roles["client"]+roles["server"])
results = en.run_command("cd /home/mdeneuville/envoy && ./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["client"]+roles["server"])

Envoy - Rebuild

In [ ]:
results = en.run_command("cd /home/mdeneuville/envoy && ./ci/run_envoy_docker.sh './ci/do_ci.sh bazel.release.server_only'", roles=roles["client"]+roles["server"])

### Envoy - FOLLOWING TIMES

In [ ]:
results = en.run_command("sudo-g5k cp -r /home/mdeneuville/envoy-docker-build/ /tmp", roles=roles["client"]+roles["server"])


### Destroy

In [ ]:
provider.destroy()
